<a href="https://colab.research.google.com/github/Mekondjo-EDSA/NLP_module/blob/main/ACLED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bokeh

In [ ]:
!pip install pandas-bokeh

In [ ]:
from pandas.errors import InvalidIndexError
import pandas as pd
import numpy as np
import pandas_bokeh
import seaborn as sns
import matplotlib.pyplot as plt
pandas_bokeh.output_notebook()
import re
import os
from datetime import timedelta
%matplotlib inline


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/acled_data_ukraine.csv', error_bad_lines=False, engine="python")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
Skipping line 674161: unexpected end of data


In [ ]:
df.head()

,data_id,iso,event_id_cnty,event_id_no_cnty,event_date,year,time_precision,event_type,sub_event_type,actor1,...,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,timestamp,iso3
0,9049601,376,ISR12000,12000.0,08 April 2022,2022,1,Violence against civilians,Attack,Unidentified Armed Group (Palestine),...,Tel Aviv,32.0809,34.7806,2,Haaretz; N12,National,"On 8 April 2022, a Palestinian militant opened...",3,1649690493,ISR
1,9049602,376,ISR12001,12001.0,08 April 2022,2022,1,Battles,Armed clash,Unidentified Armed Group (Palestine),...,Jaffa,32.0495,34.7579,2,N12; Haaretz,National,"On 8 April 2022, after a ten hour a manhunt co...",3,1649690493,ISR
2,9049629,275,PSE14519,14519.0,08 April 2022,2022,1,Protests,Peaceful protest,Protesters (Palestine),...,Al Quds - Old City,31.7767,35.2342,1,Dunia Al Watan; Palestine News and Information...,National,"On 8 April 2022, thousands of Palestinian and ...",0,1649690493,PSE
3,9049630,275,PSE14525,14525.0,08 April 2022,2022,1,Riots,Mob violence,Rioters (Palestine),...,Mukhayyam al Fawwar,31.4789,35.0659,1,PLO Negotiations Affairs Department; Palestine...,Other-National,"On 8 April 2022, Palestinian rioters hurled st...",0,1649690493,PSE
4,9049632,275,PSE14529,14529.0,08 April 2022,2022,1,Riots,Mob violence,Rioters (Israel),...,Bayt Furik,32.1769,35.3354,2,PLO Negotiations Affairs Department; Palestine...,Other-National,"On 8 April 2022, Israeli settlers hurled stone...",0,1649690493,PSE


In [ ]:
df.columns

Index(['data_id', 'iso', 'event_id_cnty', 'event_id_no_cnty', 'event_date',
       'year', 'time_precision', 'event_type', 'sub_event_type', 'actor1',
       'assoc_actor_1', 'inter1', 'actor2', 'assoc_actor_2', 'inter2',
       'interaction', 'region', 'country', 'admin1', 'admin2', 'admin3',
       'location', 'latitude', 'longitude', 'geo_precision', 'source',
       'source_scale', 'notes', 'fatalities', 'timestamp', 'iso3'],
      dtype='object')

In [ ]:
df.shape

(674159, 31)

In [ ]:
df.isnull().sum()

data_id                  0
iso                      0
event_id_cnty            0
event_id_no_cnty         0
event_date               0
year                     0
time_precision           0
event_type               0
sub_event_type           0
actor1                   0
assoc_actor_1       369541
inter1                   0
actor2              395686
assoc_actor_2       594750
inter2                   0
interaction              0
region                   0
country                  0
admin1                  81
admin2               21394
admin3              385845
location                 0
latitude                 0
longitude                0
geo_precision            0
source                   0
source_scale             0
notes                    0
fatalities               0
timestamp                0
iso3                     0
dtype: int64

In [ ]:
df.country.value_counts()

India            46848
United States    40710
Mexico           36386
Syria            31964
Ukraine          29182
                 ...  
Kiribati             1
Micronesia           1
Cook Islands         1
Montserrat           1
Brunei               1
Name: country, Length: 229, dtype: int64

In [ ]:
data = df[df['country']=='Ukraine']

In [ ]:
data.head()

,data_id,iso,event_id_cnty,event_id_no_cnty,event_date,year,time_precision,event_type,sub_event_type,actor1,...,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,timestamp,iso3
370,9058911,804,UKR55353,55353.0,08 April 2022,2022,1,Strategic developments,Other,Civilians (Ukraine),...,Kramatorsk,48.7306,37.5816,1,24 Channel; Suspilne Media,National,"Displacement: On 8 April 2022, around 2500 peo...",0,1649784778,UKR
371,9058912,804,UKR55354,55354.0,08 April 2022,2022,1,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Russia (2000-),...,Kramatorsk,48.7306,37.5816,1,Unian; Reuters; AFP; 24 Channel; RFE/RL; DPR A...,Other-National,"On 8 April 2022, Russian forces fired two Toch...",57,1649784778,UKR
372,9058913,804,UKR55355,55355.0,08 April 2022,2022,1,Battles,Armed clash,Military Forces of Russia (2000-) Air Force,...,Slovyansk,48.8533,37.6065,1,National Corps; Suspilne Media,Other-National,"On 8 April 2022, Russian forces shelled, likel...",0,1649784778,UKR
373,9058914,804,UKR55356,55356.0,08 April 2022,2022,1,Battles,Armed clash,Military Forces of Ukraine (2019-),...,Mariupol,47.1298,37.5710,1,24 Channel; Ukrinform; Flot 2017; Novoye Vremy...,Local partner-New media,"On 8 April 2022, Russian forces and Donetsk re...",4,1649784778,UKR
374,9058915,804,UKR55358,55358.0,08 April 2022,2022,1,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Russia (2000-),...,Avdiivka,48.1394,37.7497,1,24 Channel; Segodnya; Novoye Vremya Ukraine; U...,National,"On 8 April 2022, in the morning, Russian force...",5,1649784778,UKR


In [ ]:
data.shape

(29182, 31)

In [ ]:
data['event_type'].value_counts()

Explosions/Remote violence    15086
Battles                        9468
Protests                       3528
Strategic developments          597
Violence against civilians      338
Riots                           165
Name: event_type, dtype: int64

In [ ]:
data.drop(columns=['iso', 'iso3', 'country', 'region', 'year'])
data['month'] = pd.DatetimeIndex(data['event_date']).month
data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,data_id,iso,event_id_cnty,event_id_no_cnty,event_date,year,time_precision,event_type,sub_event_type,actor1,...,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,timestamp,iso3,month
370,9058911,804,UKR55353,55353.0,08 April 2022,2022,1,Strategic developments,Other,Civilians (Ukraine),...,48.7306,37.5816,1,24 Channel; Suspilne Media,National,"Displacement: On 8 April 2022, around 2500 peo...",0,1649784778,UKR,4
371,9058912,804,UKR55354,55354.0,08 April 2022,2022,1,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Russia (2000-),...,48.7306,37.5816,1,Unian; Reuters; AFP; 24 Channel; RFE/RL; DPR A...,Other-National,"On 8 April 2022, Russian forces fired two Toch...",57,1649784778,UKR,4
372,9058913,804,UKR55355,55355.0,08 April 2022,2022,1,Battles,Armed clash,Military Forces of Russia (2000-) Air Force,...,48.8533,37.6065,1,National Corps; Suspilne Media,Other-National,"On 8 April 2022, Russian forces shelled, likel...",0,1649784778,UKR,4
373,9058914,804,UKR55356,55356.0,08 April 2022,2022,1,Battles,Armed clash,Military Forces of Ukraine (2019-),...,47.1298,37.5710,1,24 Channel; Ukrinform; Flot 2017; Novoye Vremy...,Local partner-New media,"On 8 April 2022, Russian forces and Donetsk re...",4,1649784778,UKR,4
374,9058915,804,UKR55358,55358.0,08 April 2022,2022,1,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Russia (2000-),...,48.1394,37.7497,1,24 Channel; Segodnya; Novoye Vremya Ukraine; U...,National,"On 8 April 2022, in the morning, Russian force...",5,1649784778,UKR,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673868,6301668,804,UKR24927,24927.0,03 September 2019,2019,1,Explosions/Remote violence,Shelling/artillery/missile attack,NAF: United Armed Forces of Novorossiya,...,47.9425,37.5050,2,OSCE SMM-Ukraine,Other,"On 03 September, the OSCE SMM observed a total...",0,1618584320,UKR,9
673928,7224974,804,UKR24935,24935.0,03 September 2019,2019,1,Explosions/Remote violence,Shelling/artillery/missile attack,NAF: United Armed Forces of Novorossiya,...,47.0987,37.8199,2,OSCE SMM-Ukraine,Other,"On 03 September, the OSCE SMM observed a total...",0,1618584321,UKR,9
673929,7225092,804,UKR24933,24933.0,03 September 2019,2019,1,Battles,Armed clash,Military Forces of Ukraine (2019-),...,48.3133,38.0416,2,OSCE SMM-Ukraine,Other,"On 03 September, the OSCE SMM observed a total...",0,1618584321,UKR,9
673930,7225093,804,UKR24934,24934.0,03 September 2019,2019,1,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Ukraine (2019-),...,48.3798,38.0067,2,OSCE SMM-Ukraine,Other,"On 03 September, the OSCE SMM observed a total...",0,1618584321,UKR,9


In [ ]:
temp = data[['event_type', 'month', 'fatalities']]
temp = temp.groupby(['month', 'event_type']).sum()
temp = temp.reset_index()
temp = temp.pivot_table(index='month', columns ='event_type')
temp.columns = temp.columns.droplevel(0)
temp.columns.name = None
temp = temp.reset_index()
temp

,month,Battles,Explosions/Remote violence,Protests,Riots,Strategic developments,Violence against civilians
0,1,11,16,0,0,0,1
1,2,215,207,0,0,0,50
2,3,1923,1735,4,0,0,557
3,4,248,186,0,0,0,8
4,5,9,22,0,0,0,2
5,6,17,14,0,0,0,1
6,7,8,9,0,0,0,0
7,8,8,16,0,0,0,0
8,9,16,19,0,0,0,2
9,10,16,6,0,0,0,4


In [ ]:
p_stacked_hbar = temp.plot_bokeh.barh(
    x = 'month',
    stacked = True,
    xlabel = 'fatalities',
    title = '', 
    alpha = 0.6,
    legend = 'top_right',
    show_figure = True)

In [ ]:
temp1 = temp.filter(items =['month', 'Violence against civilians'])
civilian_fatalities = temp1.plot_bokeh.bar(
    x = 'month',
    stacked = False,
    xlabel = 'Fatalities due to violent action against civilians',
    title = 'Number of civilian fatalities by month', 
    alpha = 0.6,
    legend = None,
    show_figure = True)

In [ ]:
va_df = data[['actor1', 'event_type']]
va_df = va_df[va_df['event_type']=='Violence against civilians']
va_df = va_df.groupby('actor1').count().reset_index()
va_df = va_df.sort_values('event_type', ascending = False).head(10)
va_df = va_df.rename(columns={'event_type':'Count of violent acts against civilians'})
actors = va_df.plot_bokeh.bar(
    figsize=(700, 900),
    vertical_xlabel = True, 
    x = 'actor1',
    stacked = False,
    xlabel = 'perpetrators',
    ylabel = 'Violent acts against civilians',
    title = '', 
    alpha = 0.6,
    legend = None,
    show_figure = True)

In [ ]:
lva_df = data[['location', 'event_type']]
lva_df = lva_df[lva_df['event_type']=='Violence against civilians']
lva_df = lva_df.groupby('location').count().reset_index()
lva_df = lva_df.sort_values('event_type', ascending = False).head(10)
lva_df = lva_df.rename(columns={'event_type':'Count of violent acts against civilians'})
loactions = lva_df.plot_bokeh.bar(
    figsize=(700, 450),
    vertical_xlabel = True, 
    x = 'location',
    stacked = False,
    xlabel = 'locations',
    ylabel = 'Violent acts against civilians',
    title = '', 
    alpha = 0.6,
    legend = None,
    show_figure = True)

In [ ]:
!pip install gdelt

In [ ]:
import gdelt 

In [ ]:
# Version 2 queries
gd1 = gdelt.gdelt(version=1)

# pull events table, range, output to json format
results = gd1.Search(['2022 Mar 01','2022 Mar 31'],coverage=True,table='events')
print(len(results))





In [ ]:
results.head()

,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,Actor2Geo_FeatureID,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
0,1034828466,20210320,202103,2021,2021.2192,BRB,BARBADOS,BRB,NaN,NaN,...,BB,1,Barbados,BB,BB,13.166667,-59.533333,BB,20220320,https://www.msn.com/en-us/news/newsworld/princ...
1,1034828467,20210320,202103,2021,2021.2192,BUS,AEROSPACE,NaN,NaN,NaN,...,CA,1,Canada,CA,CA,60.000000,-96.0,CA,20220320,https://siouxcityjournal.com/news/science/cali...
2,1034828468,20210320,202103,2021,2021.2192,BUS,AEROSPACE,NaN,NaN,NaN,...,CA,1,Canada,CA,CA,60.000000,-96.0,CA,20220320,https://siouxcityjournal.com/news/science/cali...
3,1034828469,20210320,202103,2021,2021.2192,USA,UNITED STATES,USA,NaN,NaN,...,NaN,3,"Cedar Rapids, Iowa, United States",US,USIA,42.008300,-91.6441,465941,20220320,https://www.oskaloosa.com/news/national_news/l...
4,1034828470,20210320,202103,2021,2021.2192,USA,UNITED STATES,USA,NaN,NaN,...,1567416,3,"Kenosha, Wisconsin, United States",US,USWI,42.584700,-87.8212,1567416,20220320,https://isp.netscape.com/news/story/0001/20220...


In [ ]:
results.shape

(3910990, 59)

In [ ]:
results.columns

Index(['GLOBALEVENTID', 'SQLDATE', 'MonthYear', 'Year', 'FractionDate',
       'Actor1Code', 'Actor1Name', 'Actor1CountryCode', 'Actor1KnownGroupCode',
       'Actor1EthnicCode', 'Actor1Religion1Code', 'Actor1Religion2Code',
       'Actor1Type1Code', 'Actor1Type2Code', 'Actor1Type3Code', 'Actor2Code',
       'Actor2Name', 'Actor2CountryCode', 'Actor2KnownGroupCode',
       'Actor2EthnicCode', 'Actor2Religion1Code', 'Actor2Religion2Code',
       'Actor2Type1Code', 'Actor2Type2Code', 'Actor2Type3Code', 'IsRootEvent',
       'EventCode', 'CAMEOCodeDescription', 'EventBaseCode', 'EventRootCode',
       'QuadClass', 'GoldsteinScale', 'NumMentions', 'NumSources',
       'NumArticles', 'AvgTone', 'Actor1Geo_Type', 'Actor1Geo_FullName',
       'Actor1Geo_CountryCode', 'Actor1Geo_ADM1Code', 'Actor1Geo_Lat',
       'Actor1Geo_Long', 'Actor1Geo_FeatureID', 'Actor2Geo_Type',
       'Actor2Geo_FullName', 'Actor2Geo_CountryCode', 'Actor2Geo_ADM1Code',
       'Actor2Geo_Lat', 'Actor2Geo_Long', 'Actor

In [ ]:
# Version 2 queries
gd2 = gdelt.gdelt(version=2)

# Single 15 minute interval pull, output to json format with mentions table
mentions = gd2.Search('2016 Mar 15',table='mentions',output='json')
print(len(mentions))

3720467


/usr/local/lib/python3.7/dist-packages/gdelt/base.py:623: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  results = _mp_worker(self.download_list, proxies=self.proxies)


In [ ]:
mentions

In [ ]:
gs = gdelt.gdelt(version=2)

In [ ]:
df=gs.Search(date=['2022 03 01','2022 03 31'],table='gkg',output='df',normcols=True)